# 11、PyTorch中如何进行向量微分、矩阵微分与计算雅克比行列式

## 雅可比矩阵用法

In [7]:
import torch
from torch.autograd.functional import jacobian

In [ ]:
def func(x):
    return x.exp().sum(dim=1)


x = torch.randn(2, 3)
y = func(x)
x, y

(tensor([[-1.6274, -0.6187,  1.4563],
         [-1.0267, -0.0240,  0.5119]]),
 tensor([5.0250, 3.0029]))

In [9]:
# y 对于 x 的雅可比矩阵
jacobian(func, x)
# [[[0.1964, 0.5387, 4.2899], # y[0] 来自第一行的偏导
# [0.0000, 0.0000, 0.0000]], # y[0] 来自第二行的偏导是0

# [[0.0000, 0.0000, 0.0000], # y[1] 来自第一行的偏到是0
# [0.3582, 0.9763, 1.6685]]]

tensor([[[0.1964, 0.5387, 4.2899],
         [0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000],
         [0.3582, 0.9763, 1.6685]]])

In [ ]:
from torch import Tensor


aaa: Tensor = torch.randn(3)
print(aaa)


def func(x):
    return x + aaa


x: Tensor = torch.randn(3)
y: Tensor = func(x)
x, y

tensor([1.2148, 0.5800, 0.1546])


(tensor([-2.1449, -0.3914,  1.0567]), tensor([-0.9301,  0.1886,  1.2113]))

In [11]:
jacobian(func, x)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

### 使用backward函数求梯度

In [ ]:
x = torch.randn(3, requires_grad=True)
y = func(x)
# y.backward() # 报错
y.backward(torch.ones_like(y))  # 存在一个临时的张量，这里是 偏t再偏y
print(x.grad)

tensor([1., 1., 1.])


### 使用雅可比矩阵求梯度

In [15]:
jcb = jacobian(func, x)
torch.ones_like(y) @ jcb

tensor([1., 1., 1.])

## 矩阵到矩阵之间求导

In [30]:
a = torch.randn(2, 3, requires_grad=True)
b = torch.randn(3, 2, requires_grad=True)
c = a @ b
print(a, b, sep="\n")
# c.backward() # 报错，必须是标量
c.backward(torch.ones_like(c))
a.grad, a.grad.shape, b.grad, b.grad.shape

tensor([[-0.4925,  0.6456, -0.7058],
        [-0.8978,  0.1272, -0.1797]], requires_grad=True)
tensor([[-0.2775,  0.6865],
        [ 0.7276, -0.6722],
        [ 1.6039, -1.2717]], requires_grad=True)


(tensor([[0.4090, 0.0554, 0.3322],
         [0.4090, 0.0554, 0.3322]]),
 torch.Size([2, 3]),
 tensor([[-1.3903, -1.3903],
         [ 0.7727,  0.7727],
         [-0.8856, -0.8856]]),
 torch.Size([3, 2]))

In [34]:
# 使用雅可比矩阵
def funcb(a):
    return a @ b


# 第一行的梯度：
jcb = jacobian(funcb, a[0, :])
c = funcb(a[0, :])
print(torch.ones_like(c) @ jcb)  # 实际值为 b 的每一行加起来
# 第二行的梯度：
c = funcb(a[1, :])
jcb = jacobian(funcb, a[1, :])
print(torch.ones_like(c) @ jcb)

tensor([0.4090, 0.0554, 0.3322])
tensor([0.4090, 0.0554, 0.3322])


In [36]:
# 使用雅可比矩阵
def funca(b):
    return a @ b


# 第一行的梯度：
jcb = jacobian(funca, b[:, 0])
c = funcb(b[:, 0])
print(torch.ones_like(c) @ jcb)  # 实际值为 a 的每一列加起来
# 第二行的梯度：
c = funcb(b[:, 1])
jcb = jacobian(funca, b[:, 1])
print(torch.ones_like(c) @ jcb)

tensor([-1.3903,  0.7727, -0.8856])
tensor([-1.3903,  0.7727, -0.8856])
